In [ ]:
from src.drawProcessor     import drawProcessor
from src.drawTwoProcessors import drawTwoProcessors

## Task 1: Attack baseline processor with Spectre

The code below shows an spectre attack on a baseline out-of-order processor. We will go through the following (micro-)architecture in the class, assisted by file `1-howToUseThisTool.ipynb`:

- You provide a program with:
  - 4 types of instructions: ALU, Branch, Load, NOP
  - 8 registers: r0-r7
    - r0 is constant 0
    - r7 is also named as rSec
- ROB has infinite entries
- ALU has 4 ports, operating in parallel
- Memory has 4 entries
  - L1 can cache all 4 entries. Hit takes 1 cycle
  - On L1 miss, wait 3 cycles in MSHR


### 1-1: Spectre attack works

In [ ]:
Spectre = [
    {"dest": 1, "opcode": "ALU" , "src"   : 0, "port": 0, "latency": 4, "result": 0, "name": "delay"},
    {           "opcode": "BREZ", "src"   : 1, "offset": 6,                          "name": "br"},
    {"dest": 0, "opcode": "LOAD", "src"   : 7,                                       "name": "Tx"},
    {           "opcode": "NOP" ,                                                    "name": "nop"},
    {           "opcode": "NOP" ,                                                    "name": "nop"},
    {           "opcode": "NOP" ,                                                    "name": "nop"},
    {           "opcode": "NOP" ,                                                    "name": "nop"},
    {"dest": 0, "opcode": "LOAD", "srcImm": 0,                                       "name": "Rx"},
]
drawProcessor(
    imem=Spectre,
    r7=0,
    l1ValidArray=[False, False, False, False],
).getDraw().display_inline()

### 1-2: Show 2 simulations

You can also use `drawTwoProcessors()` function and provide `r7Pair=(0, 1)` to show 2 simulations together. This is helpful to observe the effect of secrets.

In [ ]:
drawTwoProcessors(
    imem=Spectre,
    r7Pair=(0, 1),
    l1ValidArray=[False, False, False, False],
).getDraw().display_inline()

## Task 2: Attack InvisiSpec with "speculative interference attack"

Add an argument `defense="InvisiSpec"`. This defense prevents spculative load from updating L1 state.


### 2-1: Spectre is defended by InvisiSpec defense

In [ ]:
drawTwoProcessors(
    imem=Spectre,
    r7Pair=(0, 1),
    l1ValidArray=[False, False, False, False],
    defense="InvisiSpec",
).getDraw().display_inline()

### 2-2: Attack InvisiSpec, variant 1, use MSHR contention

In [ ]:
## Task 2-2: Solution.
attack2_2 = [
    {"dest": 2, "opcode": "ALU" , "src": 0, "port": 0, "latency": 4, "result": 2, "name": "dly"},
    {"dest": 0, "opcode": "LOAD", "src": 2,                                       "name": "Rx"},
    {           "opcode": "BREZ", "src": 1, "offset": 2,                          "name": "Br"},
    {"dest": 0, "opcode": "LOAD", "src": 7,                                       "name": "Tx"},
]
attack2_2_l1ValidArray = [True, False, False, False]

drawTwoProcessors(
    imem=attack2_2,
    r7Pair=(0, 1),
    l1ValidArray=attack2_2_l1ValidArray,
    defense="InvisiSpec",
).getDraw().display_inline()

### 2-3: Attack InvisiSpec, variant 2, use ALU port contention

In [ ]:
## Task 2-3: Solution.
attack2_3 = [
    {"dest": 2, "opcode": "ALU" , "src": 0, "port": 0, "latency": 5, "result": 2, "name": "dly"},
    {"dest": 0, "opcode": "ALU" , "src": 2, "port": 2, "latency": 1, "result": 0, "name": "Rx"},
    {           "opcode": "BREZ", "src": 1, "offset": 3,                          "name": "Br"},
    {"dest": 3, "opcode": "LOAD", "src": 7,                                       "name": "Sec"},
    {"dest": 0, "opcode": "ALU" , "src": 3, "port": 2, "latency": 2, "result": 0, "name": "Tx"},
]
attack2_3_l1ValidArray = [True, False, False, False]

drawTwoProcessors(
    imem=attack2_3,
    r7Pair=(0, 1),
    l1ValidArray=attack2_3_l1ValidArray,
    defense="InvisiSpec",
).getDraw().display_inline()

## Task 3: Attack GhostMinion with a new variant of "speculative interference attack"

Add an argument `defense="GhostMinion"`. This defense further prevents younger instructions from interfering older instructions.


### 3-1: Previous attacks 2-2 and 2-3, are defended by GhostMinion defense

In [ ]:
drawTwoProcessors(
    imem=attack2_2,
    r7Pair=(0, 1),
    l1ValidArray=attack2_2_l1ValidArray,
    defense="GhostMinion",
).getDraw().display_inline()

In [ ]:
drawTwoProcessors(
    imem=attack2_3,
    r7Pair=(0, 1),
    l1ValidArray=attack2_3_l1ValidArray,
    defense="GhostMinion",
).getDraw().display_inline()

### 3-2: A new variant of "speculative interference attack" works

In [ ]:
## Task 3-2: Solution.
attack3_2 = [
    {"dest": 1, "opcode": "ALU" , "src"   : 0, "port": 0, "latency": 1, "result": 0, "name": "dlyBr"},
    {           "opcode": "BREZ", "src"   : 1, "offset": 3,                          "name": "Br"},
    {"dest": 0, "opcode": "LOAD", "src"   : 7,                                       "name": "Tx"},
    {           "opcode": "NOP" ,                                                    "name": "nop"},
    {"dest": 0, "opcode": "LOAD", "srcImm": 2,                                       "name": "Rx"},
]
attack3_2_l1ValidArray = [True, False, False, False]

drawTwoProcessors(
    imem=attack3_2,
    r7Pair=(0, 1),
    l1ValidArray=attack3_2_l1ValidArray,
    defense="GhostMinion",
).getDraw().display_inline()